In [1]:
# Append 'src' directory to import modules from notebooks directory
#################################
import os,sys
src_dir = os.path.join(os.getcwd(), os.pardir)
sys.path.append(src_dir)
##################################

In [2]:
import matplotlib.pyplot as plt
from scipy.io import loadmat
import tensorflow as tf
import numpy as np
from PIL import Image
from src.PatchMatch import PatchMatchCuda as PatchMatchOrig


In [3]:
def normalize_feat_map(feat_map):
    """
    Normalize the feature map along the channels dimension
    
    feat_map is a numpy array with channels along the 2nd dimension
    """
    return feat_map/np.linalg.norm(feat_map,axis=(2),keepdims=True)

In [4]:
mat = loadmat('../weights/imagenet-vgg-verydeep-19.mat')['layers']

In [5]:
convs = []
biases = []


for i in range(0,50):
    try:
        conv = mat[0][i][0][0][2][0][0] 
        bias = mat[0][i][0][0][2][0][1]
        assert type(conv) is np.ndarray
        assert type(bias) is np.ndarray
        convs.append(conv)
        biases.append(bias)
    except:
        pass

In [6]:
def prepare_vgg():
        
    with tf.variable_scope("VGG"):

        img = tf.placeholder(shape=[None,None,None,3],dtype=tf.float32,name='Input')
        
        with tf.variable_scope("Preprocess"):
            r,g,b = tf.unstack(img,axis=-1)
            
            b = b - 103.939
            g = g - 116.779
            r = r - 123.68
            
            img = tf.stack([b,g,r],axis=3)
        

        with tf.variable_scope("Conv1"):
            conv_1_1 = tf.layers.conv2d(inputs=img,
                                        filters=64,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[0]),
                                        bias_initializer=tf.constant_initializer(biases[0]))

            conv_1_2 = tf.layers.conv2d(inputs=conv_1_1,
                                        filters=64,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="2",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[1]),
                                        bias_initializer=tf.constant_initializer(biases[1]))

            mpool_1 = tf.layers.max_pooling2d(inputs=conv_1_2, padding='SAME', name='pool',pool_size=2,strides=2)

        with tf.variable_scope("Conv2"):
            conv_2_1 = tf.layers.conv2d(inputs=mpool_1,
                                        filters=128,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[2]),
                                        bias_initializer=tf.constant_initializer(biases[2]))

            conv_2_2 = tf.layers.conv2d(inputs=conv_2_1,
                                        filters=128,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="2",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[3]),
                                        bias_initializer=tf.constant_initializer(biases[3]))

            mpool_2 = tf.layers.max_pooling2d(inputs=conv_2_2, padding='SAME', name='pool',pool_size=2,strides=2)


        with tf.variable_scope("Conv3"):
            conv_3_1 = tf.layers.conv2d(inputs=mpool_2,
                                        filters=256,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[4]),
                                        bias_initializer=tf.constant_initializer(biases[4]))

            conv_3_2 = tf.layers.conv2d(inputs=conv_3_1,
                                        filters=256,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="2",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[5]),
                                        bias_initializer=tf.constant_initializer(biases[5]))

            conv_3_3 = tf.layers.conv2d(inputs=conv_3_2,
                                        filters=256,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="3",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[6]),
                                        bias_initializer=tf.constant_initializer(biases[6]))

            conv_3_4 = tf.layers.conv2d(inputs=conv_3_3,
                                        filters=256,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="4",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[7]),
                                        bias_initializer=tf.constant_initializer(biases[7]))

            mpool_3 = tf.layers.max_pooling2d(inputs=conv_3_4, padding='SAME', name='pool',pool_size=2,strides=2)



        with tf.variable_scope("Conv4"):
            conv_4_1 = tf.layers.conv2d(inputs=mpool_3,
                                        filters=512,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[8]),
                                        bias_initializer=tf.constant_initializer(biases[8]))

            conv_4_2 = tf.layers.conv2d(inputs=conv_4_1,
                                        filters=512,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="2",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[9]),
                                        bias_initializer=tf.constant_initializer(biases[9]))

            conv_4_3 = tf.layers.conv2d(inputs=conv_4_2,
                                        filters=512,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="3",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[10]),
                                        bias_initializer=tf.constant_initializer(biases[10]))

            conv_4_4 = tf.layers.conv2d(inputs=conv_4_3,
                                        filters=512,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="4",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[11]),
                                        bias_initializer=tf.constant_initializer(biases[11]))

            mpool_4 = tf.layers.max_pooling2d(inputs=conv_4_4, padding='SAME', name='pool',pool_size=2,strides=2)



        with tf.variable_scope("Conv5"):
            conv_5_1 = tf.layers.conv2d(inputs=mpool_4,
                                        filters=512,
                                        kernel_size=3
                                        ,padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[12]),
                                        bias_initializer=tf.constant_initializer(biases[12]))

        return conv_5_1

In [46]:
def deconv_block_4(conv5_1_True_NP,sess):
        
        with tf.variable_scope("Deconv_Block_4",reuse=tf.AUTO_REUSE):
            
            conv5_1_True_NP = np.expand_dims(conv5_1_True_NP,axis=0)
            
            conv_5_1_True = tf.get_variable(name='Conv5_1_True',shape=[1,
                                                               conv5_1_True_NP.shape[1],
                                                               conv5_1_True_NP.shape[2],
                                                               512])
            
            conv_4_1_Noise = tf.get_variable(name='conv_3_1_Noise',shape=[1,
                                                             conv5_1_True_NP.shape[1]*2,
                                                             conv5_1_True_NP.shape[2]*2,
                                                             512])
            conv_4_1_Noise_NP = np.random.randn(1,
                                    conv5_1_True_NP.shape[1]*2,
                                    conv5_1_True_NP.shape[2]*2,
                                    512)*127

            with tf.variable_scope("Conv4",reuse=tf.AUTO_REUSE):

                conv_4_2 = tf.layers.conv2d(inputs=conv_4_1_Noise,
                                            filters=512,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="2",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[9]),
                                            bias_initializer=tf.constant_initializer(biases[9]))

                conv_4_3 = tf.layers.conv2d(inputs=conv_4_2,
                                            filters=512,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="3",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[10]),
                                            bias_initializer=tf.constant_initializer(biases[10]))

                conv_4_4 = tf.layers.conv2d(inputs=conv_4_3,
                                            filters=512,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="4",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[11]),
                                            bias_initializer=tf.constant_initializer(biases[11]))

                mpool_4 = tf.layers.max_pooling2d(inputs=conv_4_4, padding='SAME', name='pool',pool_size=2,strides=2)

            with tf.variable_scope("Conv5",reuse=False):
                conv_5_1 = tf.layers.conv2d(inputs=mpool_4,
                                            filters=512,
                                            kernel_size=3
                                            ,padding='SAME',
                                            name="1",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[12]),
                                            bias_initializer=tf.constant_initializer(biases[12]))

###################################


# Assignment ops. These are required to set the noise and True conv2_1 output to optimize for. 
# This prevent constant gpu to cpu transfer , resulting in higher performance

        conv_4_1_assign_op = tf.assign(conv_4_1_Noise,
                                       conv_4_1_Noise_NP,
                                       validate_shape=False)
        
        conv_5_1_True_assign_op = tf.assign(conv_5_1_True,
                                            conv5_1_True_NP,
                                            validate_shape=False)
        


################################

        loss_op = tf.losses.mean_squared_error(conv_5_1_True,
                                               conv_5_1)
        optim_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op,var_list=[conv_4_1_Noise])


        sess.run(tf.global_variables_initializer())
        
        sess.run([conv_4_1_assign_op,
                  conv_5_1_True_assign_op])
        

        
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(
          var_list=[conv_4_1_Noise],
          loss = loss_op, method='L-BFGS-B',
          options={'disp': True})
        optimizer.minimize(sess)
        
        return sess.run(conv_4_1_Noise)

In [47]:
def deconv_block_3(conv4_1_True_NP,sess):
        
        with tf.variable_scope("Deconv_Block_3",reuse=tf.AUTO_REUSE):
            
            conv4_1_True_NP = np.expand_dims(conv4_1_True_NP,axis=0)
            conv_4_1_True = tf.get_variable(name='Conv4_1_True',shape=[1,
                                                               conv4_1_True_NP.shape[1],
                                                               conv4_1_True_NP.shape[2],
                                                               512])
            
            conv_3_1_Noise = tf.get_variable(name='conv_3_1_Noise',shape=[1,
                                                             conv4_1_True_NP.shape[1]*2,
                                                             conv4_1_True_NP.shape[2]*2,
                                                             256])
            conv_3_1_Noise_NP = np.random.randn(1,
                                    conv4_1_True_NP.shape[1]*2,
                                    conv4_1_True_NP.shape[2]*2,
                                    256)*127



            with tf.variable_scope("Conv3",reuse=tf.AUTO_REUSE):

                conv_3_2 = tf.layers.conv2d(inputs=conv_3_1_Noise,
                                            filters=256,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="2",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[5]),
                                            bias_initializer=tf.constant_initializer(biases[5]))

                conv_3_3 = tf.layers.conv2d(inputs=conv_3_2,
                                            filters=256,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="3",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[6]),
                                            bias_initializer=tf.constant_initializer(biases[6]))

                conv_3_4 = tf.layers.conv2d(inputs=conv_3_3,
                                            filters=256,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="4",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[7]),
                                            bias_initializer=tf.constant_initializer(biases[7]))

                mpool_3 = tf.layers.max_pooling2d(inputs=conv_3_4, padding='SAME', name='pool',pool_size=2,strides=2)



            with tf.variable_scope("Conv4",reuse=tf.AUTO_REUSE):
                conv_4_1 = tf.layers.conv2d(inputs=mpool_3,
                                            filters=512,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="1",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[8]),
                                            bias_initializer=tf.constant_initializer(biases[8]))

###################################


# Assignment ops. These are required to set the noise and True conv2_1 output to optimize for. 
# This prevent constant gpu to cpu transfer , resulting in higher performance

        conv_3_1_assign_op = tf.assign(conv_3_1_Noise,
                                       conv_3_1_Noise_NP,
                                       validate_shape=False)
        
        conv_4_1_True_assign_op = tf.assign(conv_4_1_True,
                                            conv4_1_True_NP,
                                            validate_shape=False)
        


################################

        loss_op = tf.losses.mean_squared_error(conv_4_1_True,
                                               conv_4_1)
        optim_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op,var_list=[conv_3_1_Noise])


        sess.run(tf.global_variables_initializer())
        
        sess.run([conv_3_1_assign_op,
                  conv_4_1_True_assign_op])
        

        
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(
          var_list=[conv_3_1_Noise],
          loss = loss_op, method='L-BFGS-B',
          options={'disp': True})
        optimizer.minimize(sess)
        
        return sess.run(conv_3_1_Noise)

In [48]:
def deconv_block_2(conv3_1_True_NP,sess):

    with tf.variable_scope("Deconv_Block_2",reuse=tf.AUTO_REUSE):
        
        conv3_1_True_NP = np.expand_dims(conv3_1_True_NP,axis=0)
        conv_3_1_True = tf.get_variable(name='Conv3_1_True',shape=[1,
                                                           conv3_1_True_NP.shape[1],
                                                           conv3_1_True_NP.shape[2],
                                                           256])
        conv_2_1_Noise = tf.get_variable(name='conv_2_1_Noise',shape=[1,
                                                         conv3_1_True_NP.shape[1]*2,
                                                         conv3_1_True_NP.shape[2]*2,
                                                         128])
        conv_2_1_Noise_NP = np.random.randn(1,
                                conv3_1_True_NP.shape[1]*2,
                                conv3_1_True_NP.shape[2]*2,
                                128)*127
################# Sub VGG###################

    


        with tf.variable_scope("Conv2",reuse=tf.AUTO_REUSE):

            conv_2_2 = tf.layers.conv2d(inputs=conv_2_1_Noise,
                                            filters=128,
                                            kernel_size=3,
                                            padding='SAME',
                                            name="2",
                                            activation=tf.nn.relu,
                                            kernel_initializer=tf.constant_initializer(convs[3]),
                                            bias_initializer=tf.constant_initializer(biases[3]))

            mpool_2 = tf.layers.max_pooling2d(inputs=conv_2_2, padding='SAME', name='pool',pool_size=2,strides=2)


        with tf.variable_scope("Conv3",reuse=tf.AUTO_REUSE):
            conv_3_1 = tf.layers.conv2d(inputs=mpool_2,
                                        filters=256,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[4]),
                                        bias_initializer=tf.constant_initializer(biases[4]))
            
            
###################################


# Assignment ops. These are required to set the noise and True conv2_1 output to optimize for. 
# This prevent constant gpu to cpu transfer , resulting in higher performance

        conv_2_1_assign_op = tf.assign(conv_2_1_Noise,
                                       conv_2_1_Noise_NP,
                                       validate_shape=False)
        
        conv_3_1_True_assign_op = tf.assign(conv_3_1_True,
                                            conv3_1_True_NP,
                                            validate_shape=False)
        


################################

        loss_op = tf.losses.mean_squared_error(conv_3_1_True,
                                               conv_3_1)
        optim_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op,var_list=[conv_2_1_Noise])


        sess.run(tf.global_variables_initializer())
        
        sess.run([conv_2_1_assign_op,
                  conv_3_1_True_assign_op])
        

        
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(
          var_list=[conv_2_1_Noise],
          loss = loss_op, method='L-BFGS-B',
          options={'disp': True})
        optimizer.minimize(sess)
        
        return sess.run(conv_2_1_Noise)

In [49]:
def deconv_block_1(conv2_1_True_NP,sess):
    with tf.variable_scope("Deconv_Block_1",reuse=tf.AUTO_REUSE):
        
        conv2_1_True_NP = np.expand_dims(conv2_1_True_NP,axis=0)
        conv_2_1_True = tf.get_variable(name='Conv2_1_True',shape=[1,
                                                           conv2_1_True_NP.shape[1],
                                                           conv2_1_True_NP.shape[2],
                                                           128])
        conv_1_1_Noise = tf.get_variable(name='conv_1_1_Noise',shape=[1,
                                                         conv2_1_True_NP.shape[1]*2,
                                                         conv2_1_True_NP.shape[2]*2,
                                                         64])
        conv_1_1_Noise_NP = np.random.randn(1,
                                conv2_1_True_NP.shape[1]*2,
                                conv2_1_True_NP.shape[2]*2,
                                64)*127
################# Sub VGG###################
        with tf.variable_scope("Conv1",reuse=tf.AUTO_REUSE):
            conv_1_2 = tf.layers.conv2d(inputs=conv_1_1_Noise,
                                        filters=64,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="2",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[1]),
                                        bias_initializer=tf.constant_initializer(biases[1]))

            mpool_1 = tf.layers.max_pooling2d(inputs=conv_1_2,
                                              padding='SAME',
                                              name='pool',
                                              pool_size=2,
                                              strides=2)

        with tf.variable_scope("Conv2",reuse=tf.AUTO_REUSE):
            conv_2_1 = tf.layers.conv2d(inputs=mpool_1,
                                        filters=128,
                                        kernel_size=3,
                                        padding='SAME',
                                        name="1",
                                        activation=tf.nn.relu,
                                        kernel_initializer=tf.constant_initializer(convs[2]),
                                        bias_initializer=tf.constant_initializer(biases[2]))
###################################



# Assignment ops. These are required to set the noise and True conv2_1 output to optimize for. 
# This prevent constant gpu to cpu transfer , resulting in higher performance

        conv_1_1_assign_op = tf.assign(conv_1_1_Noise,
                                       conv_1_1_Noise_NP,
                                       validate_shape=False)
        
        conv_2_1_True_assign_op = tf.assign(conv_2_1_True,
                                            conv2_1_True_NP,
                                            validate_shape=False)

################################

        loss_op = tf.losses.mean_squared_error(conv_2_1_True,
                                               conv_2_1)
        optim_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op,var_list=[conv_1_1_Noise])


        sess.run(tf.global_variables_initializer())
        
        sess.run([conv_1_1_assign_op,
                  conv_2_1_True_assign_op])
        
        optimizer = tf.contrib.opt.ScipyOptimizerInterface(
          var_list=[conv_1_1_Noise],
          loss = loss_op, method='L-BFGS-B',
          options={'disp': True})
        optimizer.minimize(sess)
        
        return sess.run(conv_1_1_Noise)

        

In [50]:
img1 = np.array(Image.open('../data/raw/ava.png').convert('RGB').resize((224,224)),dtype=np.float32)
img1 = np.expand_dims(img1,axis=0)

img2 = np.array(Image.open('../data/raw/mona.png').convert('RGB').resize((224,224)),dtype=np.float32)
img2 = np.expand_dims(img2,axis=0)


In [51]:
sess = tf.InteractiveSession(graph=tf.Graph())

In [52]:
final_conv = prepare_vgg()

In [53]:
sess.run(tf.global_variables_initializer())
output1 = sess.run('VGG/Conv5/1/Relu:0',feed_dict={'VGG/Input:0':img1})

TEST = deconv_block_4(output1.squeeze(),sess)

writer = tf.summary.FileWriter("../logs",sess.graph)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 17.311647
  Number of iterations: 175
  Number of functions evaluations: 186


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 3.889272
  Number of iterations: 249
  Number of functions evaluations: 259

In [15]:
output1.shape

(1, 56, 56, 256)

In [16]:
output = output.squeeze()
output1 = output1.squeeze()

NameError: name 'output' is not defined

In [ ]:
feata = normalize_feat_map(output)
feataa = normalize_feat_map(output)

featb = normalize_feat_map(output1)
featbb = normalize_feat_map(output1)

In [ ]:
pm5ab = PatchMatchOrig(feata,feataa,featb,featbb, 3)
# plt.imshow(pm5ab.visualize())
pm5ab.propagate(iters=5,rand_search_radius=5000)

In [ ]:
recon = pm5ab.reconstruct_image(img1)
recon = np.ndarray.astype(recon,np.uint8)


In [ ]:
plt.imshow(recon)

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(16, 512//4))
columns = 8
rows = 512//8
for i in range(1, columns*rows +1):
    img = output[:,:,i-1]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img,cmap='gray')
    plt.axis('off')
plt.show()


In [ ]:
output1.shape

In [ ]:
[conv.shape for conv in convs]

In [ ]:
output[2].shape

In [ ]:
tf.get_variable('ok',shape=[1])

In [ ]:
tf.reset_default_graph()
v = None
with tf.variable_scope("",reuse=tf.AUTO_REUSE):
    v = tf.get_variable('ok',shape=[1,2,3])
    

In [ ]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [ ]:
sess.run('Deconv_Block_1/Input:0').shape

In [ ]:
sess.run('Deconv_Block_1/Conv2_1_True:0').shape

In [ ]:
for i in range(10000):
    v = tf.Variable(name='ok',initial_value=[1,2,3])
sess.run(tf.global_variables_initializer())


In [ ]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

In [ ]:
np.random.randn(112,112,128)*10000

In [ ]:
output.squeeze().shape

In [ ]:
batch.shape

In [ ]:
plt.imshow(TEST[0,:,:,0])

In [ ]:
plt.imshow(output[:,:,2])

In [ ]:
output.shape

In [ ]:
# sess.run(tf.global_variables_initializer())
TEST.minimize(sess)

In [31]:
TEST.shape

(1, 56, 56, 256)